In [ ]:
import open3d as o3d
from open3d import *
import numpy as np
import struct
from natsort import natsorted
import os
import time
from tqdm import tqdm 
import pickle  
import icp 
import copy
from numpy.linalg import inv 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_style("whitegrid")
import glob

In [ ]:
def convert_kitti_bin_to_pcd(binFilePath):
    # Load binary point cloud
    bin_pcd = np.fromfile(binFilePath, dtype=np.float32)

    # Reshape and drop reflection values
    points = bin_pcd.reshape((-1, 4))[:, 0:3]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    # Convert to Open3D point cloud
    return pcd

input_dir = "00/"
input_files = natsorted(os.listdir(input_dir))

pcds = []

try : 
    fileObj = open('PCDs.pikle', 'rb')
    pcds = pickle.load(fileObj)
    fileObj.close()
except:
    pcds = []
    for in_file in tqdm(input_files):
        pcd = convert_kitti_bin_to_pcd(os.path.join(input_dir, in_file))
        pcd_np = np.asanyarray(pcd.points)
        pcds.append(pcd_np)
    
    fileObj = open('PCDs.pikle', 'wb')
    pickle.dump(pcds,fileObj)
    fileObj.close()

In [ ]:

for i in range(len(pcds)):
    point_cloud = pcds [i]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    #pcd = pcd.voxel_down_sample(voxel_size=0.01)
    o3d.io.write_point_cloud("./ndt_run/scan_{0}.pcd".format(i),pcd)

In [ ]:
transformations = [np.eye(4)]
world_map = []
fileObj = open('./slam_icp_result_transformation.pikle', 'rb')
tra = pickle.load(fileObj)
fileObj.close()

for index in tqdm(range(1,142)):
    mat = open("./ndt_run/transformation_{0}_{1}.txt".format(index,index-1),"r").read()
    mat = np.array(list(map(np.float64,mat.split()))).reshape(4,4)
    #mat[:3,3] = mat[:3,3] * 10
    transformations.append(mat)
    
    point_cloud = pcds[index]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    #pcd = pcd.voxel_down_sample(voxel_size=0.01)
    
    if index == 1:
        world_map = np.array(pcd.points)
        world_map = np.round(np.array(pcd.points),decimals=1)
        world_map = np.unique(world_map,axis=0)
    
    for _ in range(len(transformations)-1,-1,-1):
        pcd.transform(transformations[_])
    
    world_map = np.concatenate([world_map,np.array(pcd.points)])
    world_map = np.round(world_map,decimals=1) 
    world_map = np.unique(world_map,axis=0) 
    #world_map = np.array(pcd.points)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    world_map_pcd = o3d.geometry.PointCloud()
    world_map_pcd.points = o3d.utility.Vector3dVector(world_map)
    vis.add_geometry(world_map_pcd)
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image("./resultndt.png".format(index))    


In [ ]:
def calculate_driving_lines(world_map,transformation,name="icp"):
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    world_map_pcd = o3d.geometry.PointCloud()
    world_map_pcd.points = o3d.utility.Vector3dVector(world_map)
    vis.add_geometry(world_map_pcd)
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image("./results/result_{0}_world_map_drive_path.png".format(name))
    vis.destroy_window()
        
    Centers = []
    Centers_gt = []
    gt_file = open("./gt_pose_00.txt").readlines()


    for pc_index in tqdm(range(len(transformation))):
        point_cloud = np.array([[0,0,100],[0,0,0]])
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud)
        pcd = pcd.voxel_down_sample(voxel_size=0.1)
        
        for _ in range(pc_index-1,-1,-1):
            pcd.transform(transformation[_])
            
        Centers.append(np.asarray(pcd.points)[0][:2])
        
        world_map = np.concatenate([world_map,np.array(pcd.points)])
            
        point_cloud = np.array([[0,0,0]])
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud)
        pcd = pcd.voxel_down_sample(voxel_size=0.1)
        
        gt = gt_file[pc_index]
        gt = gt.strip()
        gt = np.array(list(map(np.float64, gt.split()))).reshape(3,4)
        gt = np.concatenate([gt,np.array([[0,0,0,1]])])    
        pcd.transform(gt)
        Centers_gt.append(np.asarray(pcd.points)[0][:2])
        
        
        vis = o3d.visualization.Visualizer()
        vis.create_window(visible=False)
        
        world_map_pcd = o3d.geometry.PointCloud()
        world_map_pcd.points = o3d.utility.Vector3dVector(world_map)
        vis.add_geometry(world_map_pcd)
        vis.poll_events()
        vis.update_renderer()
        vis.capture_screen_image("./results/result_{1}_world_map_{0}_drive_path.png".format(pc_index,name))
        vis.destroy_window()

    Centers = np.array(Centers)
    Centers = Centers/Centers.max(axis=0)
    plt.plot(Centers[:,1],Centers[:,0],label="Estimated driving line")

    Centers_gt = np.array(Centers_gt) * (-1)
    Centers_gt = Centers_gt/Centers_gt.max(axis=0)
    plt.plot(Centers_gt[:,0],Centers_gt[:,1],label="Ground truth driving line")
    plt.title("Driving lines")
    plt.ylabel("Y axis (Normalized & Aligned)")
    plt.xlabel("X axis (Normalized & Aligned)")

    plt.legend()
    plt.show()

world_map_cp = copy.deepcopy(world_map.copy())
transformation = copy.deepcopy(transformations.copy())
calculate_driving_lines(world_map,transformation,name="ndt")

In [ ]:
fileObj = open('./slam_ndt_result_transformation.pikle', 'wb')
pickle.dump(transformations,fileObj)
fileObj.close()